In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：超参数调整文本二分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_text_binary_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_text_binary_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python中的Vertex客户端库为自定义文本二元分类模型进行超参数调整。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)。您在本教程中使用的数据集版本存储在一个公共云存储桶中。训练的模型预测评论的情感是积极的还是消极的。

### 目标

在这个笔记本中，您将学习如何使用Vertex客户端库从Python脚本中在docker容器中创建一个用于自定义文本二分类模型的超参数调整作业。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线调整模型的超参数。

执行的步骤包括：

- 创建一个用于训练自定义模型的Vertex超参数调整作业。
- 调整自定义模型。
- 评估研究结果。

### 成本

本教程使用 Google Cloud (GCP) 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

重新启动内核

一旦您安装了Vertex客户端库和Google *云存储*，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*确保如果有这个选项的话，你正在GPU运行时中运行这个笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置你的谷歌云项目

**无论你的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个账户时，你会获得一个用于计算/存储成本的 $300 免费信用。

2. [确保你的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入你的项目ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于整个笔记本中的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法将多区域存储桶用于Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果你在一个直播教程会话中，你可能正在使用一个共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，你为每个实例会话创建一个时间戳，并将其附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Google Cloud Notebook**，您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户访问权限该项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**无论您使用什么笔记本环境，下面的步骤都是必需的。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要上传一个包含训练代码的 Python 包到一个云存储桶中。Vertex会从这个包中运行代码。在这个教程中，Vertex还会把训练作业产生的训练模型保存在同一个桶中。然后，您可以基于这个输出创建一个 `Endpoint` 资源，用于提供在线预测。

请设置您的云存储存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时，才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和端点服务的顶点API服务端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 容器（Docker）映像

接下来，我们将设置用于训练的Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`将是版本2.1，`1-15`将是版本1.15。以下列表显示了一些可用的预构建映像:

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 机器类型

接下来，设置用于训练的机器类型。

- 设置变量 `TRAIN_COMPUTE` 来配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU的内存为3.75GB。
     - `n1-highmem`：每个vCPU的内存为6.5GB。
     - `n1-highcpu`：每个vCPU的内存为0.9GB。
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

# 教程

现在你已经准备好开始创建你自己的超参数调整和训练自定义文本二元分类任务。

## 设置客户端

Vertex客户端库以客户端/服务器模型运行。在您的一端（Python脚本）中，您将创建一个客户端，该客户端发送请求并接收来自Vertex服务器的响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 为`Model`资源设置模型服务。
- 为超参数调整设置作业服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()

for client in clients.items():
    print(client)

调整模型 - 你好世界

有两种方法可以使用容器映像来调整超参数并训练自定义模型：

- **使用谷歌云预构建的容器**。如果使用预构建的容器，您还需要指定要安装到容器映像中的 Python 包。这个 Python 包包含了用于超参数调整和训练自定义模型的代码。

- **使用您自己的自定义容器映像**。如果使用您自己的容器，容器需要包含用于超参数调整和训练自定义模型的代码。

准备好你的超参数调整工作规范

现在你的客户准备好了，你的第一步是为你的超参数调整工作创建一个工作规范。工作规范将包括以下内容：

- `trial_job_spec`：自定义工作的规范。
  - `worker_pool_spec`：用于超参数调整的机器类型和数量的规范（单个或分布式）。
  - `python_package_spec`：要与预构建容器一起安装的Python包的规范。

- `study_spec`：调整内容的规范。
  - `parameters`：这是您将为自定义训练作业调整的超参数的规范。它将包含一个列表
  - `metrics`：这是评估每次调整试验结果的规范。

###准备您的机器规格

现在为您定制的超参数调整工作定义机器规格。这告诉 Vertex 应该为超参数调整提供什么类型的机器实例。
  - `machine_type`：要配置的 GCP 实例类型 -- 例如，n1-standard-8。
  - `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您先前设置了变量 `TRAIN_GPU != None`，则您正在使用 GPU；否则，您将使用 CPU。
  - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义超参数调整作业定义磁盘规格。这将告诉 Vertex 需要在每台机器实例中为超参数调整提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘的大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作人员池规范

接下来，您需要为您的自定义超参数调整作业定义工作人员池规范。工作人员池规范将包括以下内容：

- `replica_count`：要提供的这种机器类型的实例数量。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。

- `python_package`：要安装在 VM 实例上的 Python 训练包，以及要调用的 Python 模块，以及针对 Python 模块的命令行参数。

现在让我们更深入地了解Python软件包规范：

- `executor_image_spec`：这是为您的自定义超参数调整作业配置的docker镜像。

- `package_uris`：这是要安装在提供的实例上的python训练包的位置（URI）列表。这些位置需要在一个云存储桶中。这些可以是单个python文件或整个包的zip（归档）文件。在后一种情况下，作业服务将将内容解压缩到docker镜像中。

- `python_module`：用于运行自定义超参数调整作业的Python模块（脚本）。在这个例子中，您将调用 `trainer.task.py` -- 请注意，不需要添加 `.py` 后缀。

- `args`：要传递给相应的Python模块的命令行参数。在这个例子中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的云存储位置。告诉超参数调整脚本在哪里保存模型工件有两种方式：
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量 `DIRECT = True`），
      - 间接：服务将Cloud存储位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数（批次数）。
  - `"--distribute=" + TRAIN_STRATEGY"`：单个或分布式超参数调整所使用的超参数调整分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_imdb.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 创建研究规范

让我们从一个简单的研究开始。您将只使用一个参数 -- *学习速率*。由于只有一个参数，进行随机搜索没有太多意义。相反，我们将在一系列数值范围内进行网格搜索。

- `度量`:
 - `度量ID`: 在这个示例中，要报告的客观指标是 `'val_accuracy'`
 - `目标`: 在这个示例中，超参数调整服务将评估试验以最大化客观指标的价值。
- `参数`: 要调整的超参数的规范。
 - `参数ID`: 将传递给 Python 软件包作为命令行参数的超参数名称。
  - `比例类型`: 比例类型确定超参数调整服务在搜索搜索空间时使用的分辨率。
   - `单位线性比例`: 在搜索空间中的分辨率是相同的。
   - `单位对数比例`: 接近搜索空间底部的值更远。
   - `单位反对数比例`: 接近搜索空间顶部的值更远。
 - **搜索空间**: 在这里您将指定要选择调整的超参数的值的搜索空间。
   - `整数值规范`: 指定在 `min_value` 和 `max_value` 之间的整数范围的值。
   - `双值规范`: 指定在 `min_value` 和 `max_value` 之间的连续范围的值。
   - `离散值规范`: 指定一组值。
- `算法`: 选择每个试验的超参数值的搜索方法:
 - `网格搜索`: 组合搜索 -- 这在这个示例中使用。
 - `随机搜索`: 随机搜索。

In [ ]:
study_spec = {
    "metrics": [
        {
            "metric_id": "val_accuracy",
            "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE,
        }
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        }
    ],
    "algorithm": aip.StudySpec.Algorithm.GRID_SEARCH,
}

### 组装超参数调整作业规范

现在组装完整的自定义超参数调整规范描述：

- `display_name`：您为此自定义超参数调整作业分配的易读名称。
- `trial_job_spec`：自定义超参数调整作业的规范。
- `study_spec`：要调整的内容规范。
- `max_trial_count`：调整试验的最大数量。
- `parallel_trial_count`：要并行尝试的试验数量；否则，它们会按顺序执行。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

### 检查超参数调整包

#### 包布局

在开始超参数调整之前，您将查看一个如何为自定义超参数调整工作组装Python包。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

`setup.cfg` 和 `setup.py` 文件是安装该包到 Docker 镜像操作环境中的说明。

`trainer/task.py` 文件是执行自定义超参数调整工作的Python脚本。 *注意*，在工作组规范中引用时，我们将目录斜杠替换为点 (`trainer.task`)，并且去掉文件后缀 (`.py`)。

#### 包装配

接下来，您将组装训练包。

In [ ]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: IMDB Movie Reviews text binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration hyperparameter tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您要写超参数调整脚本task.py的内容。我不会详细解释，只是让您浏览。总结一下：

- 作为命令行参数传递试验的超参数值（`parser.add_argument（'--lr'，...）`）
- 模拟训练循环，在每次循环（epoch）中，将变量`accuracy`设定为循环迭代次数乘以学习率。
- 使用`report_hyperparameter_tuning_metric()`向超参数调整服务回报客观指标`accuracy`。

In [ ]:
%%writefile custom/trainer/task.py
# HP Tuning hello world example

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import argparse
import os
import sys
import time
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--model-dir',
                    dest='model_dir',
                    default='/tmp/saved_model',
                    type=str,
                    help='Model dir.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Instantiate the HyperTune reporting object
hpt = HyperTune()

for epoch in range(1, args.epochs+1):
    # mimic metric result at the end of an epoch
    acc = args.lr * epoch
    # save the metric result to communicate back to the HPT service
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_accuracy',
        metric_value=acc,
        global_step=epoch)
    print('epoch: {}, accuracy: {}'.format(epoch, acc))
    time.sleep(1)

将超参数调整脚本存储在您的云存储桶中

接下来，您将超参数调整文件夹打包成一个压缩的tar文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

使用 hypertune 报告试验结果

对于每个试验，您的 Python 脚本需要向超参数调整服务报告您指定为评估试验标准的客观指标。

在本示例中，您将在研究规范中指定客观指标将被报告为 `loss`。

您可以使用 `HyperTune` 报告客观指标的值。这个 Python 模块用于将关键值对发送给超参数调整服务。要在您的 Python 包中设置此报告功能，您需要添加以下三个步骤的代码：

1. 导入 HyperTune 模块：`from hypertune import HyperTune()`.
2. 在每个 epoch 结束时，使用 `hpt.report_hyperparameter_tuning_metric()` 将客观函数的当前值写入日志作为键值对。在本例中，参数为：
 - `hyperparameter_metric_tag`：要报告的客观指标的名称。该名称必须与研究规范中指定的名称相同。
 - `metric_value`：要报告给超参数服务的客观指标的值。
 - `global_step`：epoch 迭代次数，从0开始。

## 超参数调整模型

现在开始在Vertex上对您的自定义模型进行超参数调整。使用以下辅助函数`create_hyperparameter_tuning_job`，该函数接受以下参数：

- `hpt_job`：超参数调整作业的规范。

辅助函数调用作业客户端服务的`create_hyperparameter_tuning_job`方法，并传入以下参数：

- `parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `hyperparameter_tuning_job`：超参数调整作业的规范。

您将在`response`对象返回的字段中显示一些感兴趣的字段，其中最感兴趣的两个是：

`response.name`：分配给此自定义超参数调整作业的Vertex完全限定标识符。您将保存这个标识符以供后续步骤使用。

`response.state`：自定义超参数调整作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获取您创建的超参数调整作业的唯一标识符。

In [ ]:
# The full unique ID for the hyperparameter tuning job
hpt_job_id = response.name
# The short numeric ID for the hyperparameter tuning job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

获取超参数调整作业的信息

接下来，使用这个辅助函数`get_hyperparameter_tuning_job`，它接受以下参数：

- `name`：超参数调整作业的Vertex完全限定标识符。

该辅助函数调用作业客户端服务的`get_hyperparameter_tuning_job`方法，参数如下：

- `name`：超参数调整作业的Vertex完全限定标识符。

如果您记得，在调用`create_hyperparameter_tuning_job`方法时，在`response.name`字段中获得了超参数调整作业的Vertex完全限定标识符，并将该标识符保存在变量`hpt_job_id`中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

等待调整完成

对上述模型进行超参数调整可能需要20分钟以上的时间。

一旦您的模型调整完成，您可以通过将`end_time`减去`start_time`来计算实际调整模型所花费的时间。

对于您的模型，我们需要知道每次试验的保存模型的位置，Python脚本将其保存在您本地的云存储桶中的`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

现在审查研究结果。现在审查试验结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

现在看看哪个试验是最好的：

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

获取最佳模型

如果您使用了让服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件会保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

## 调整模型 - IMDB电影评论

现在您已经看到了使用一个模仿训练模型的Python包来调整超参数的整体步骤，您将为一个用于IMDB电影评论模型的自定义训练工作做一个新的超参数调整工作。

在这个例子中，您需要改变两个部分：

1. 指定用于IMDB电影评论自定义超参数调整的Python包。
2. 指定一个针对IMDB电影评论自定义超参数调整Python包中使用的超参数的研究规范。

### 创建一个研究规范

在这个研究中，您将使用随机搜索算法调整两个超参数：

- **学习率**：搜索空间是一组离散值。
- **学习率衰减**：搜索空间是在1e-6和1e-2之间的连续范围。

目标（目标）是最大化验证准确率。

您将最多运行六次试验。

In [ ]:
study_spec = {
    "metrics": [
        {"metric_id": "loss", "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE}
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
        {
            "parameter_id": "decay",
            "double_value_spec": {"min_value": 1e-6, "max_value": 1e-2},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
    ],
    "algorithm": aip.StudySpec.Algorithm.RANDOM_SEARCH,
}

### 组装超参数调整作业规范

现在组装自定义超参数调整规范的完整描述：

- `display_name`：您为此自定义超参数调整作业分配的易读名称。
- `trial_job_spec`：自定义超参数调整作业的规范。
- `study_spec`：要调整的规范。
- `max_trial_count`：调整试验的最大数量。
- `parallel_trial_count`：并行尝试的试验数量；否则，它们将按顺序执行。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

#### Task.py 内容

在下一个单元格中，您可以编写超参数调整脚本 task.py 的内容。我不会详细介绍，只是供您查阅。总结如下：

- 解析当前试验的超参数设置的命令行参数。
- 从命令行中获取保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）中加载 IMDB 电影评价数据集。
- 使用 TF.Keras 模型 API 构建一个简单的 RNN 模型。
- 在模型的编译中使用学习率和每个 Dense 和 LSTM 层的单元数超参数值。
- 编译模型（`compile()`）。
- 定义一个名为 `HPTCallback` 的回调函数，在每个 epoch 结束时获取验证损失（`on_epoch_end()`），并通过 `hpt.report_hyperparameter_tuning_metric()` 报告给超参数调整服务。
- 使用 `fit()` 方法训练模型，并指定一个回调函数，用于将验证损失报告回超参数调整服务。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for IMDB Movie Reviews
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Preparing dataset
BUFFER_SIZE = 1000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(args.units)),
        tf.keras.layers.Dense(args.units, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=args.lr, decay=args.decay),
        metrics=['accuracy'])
    return model


model = build_and_compile_rnn_model(encoder)

# Instantiate the HyperTune reporting object
hpt = HyperTune()


# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='loss',
        metric_value=logs['loss'],
        global_step=epoch)


model.fit(train_dataset, epochs=args.epochs, callbacks=[HPTCallback()])
model.save(args.model_dir)

将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成压缩的tar格式，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

#### 使用优化超参数报告试验结果

对于每个试验，您的Python脚本需要向超参数调整服务报告您指定的用于评估试验的客观指标。

在本示例中，您将在研究规范中指定客观指标将被报告为`loss`。

您可以使用`HyperTune`报告客观指标的值。这个Python模块用于向超参数调整服务通信关键/值对。要在您的Python包中设置此报告，您需要添加以下三个步骤的代码：

1. 导入HyperTune模块：`from hypertune import HyperTune()`.
2. 在每个epoch结束时，使用`hpt.report_hyperparameter_tuning_metric()`将客观函数的当前值作为键/值对写入日志。在这个示例中，参数为：
 - `hyperparameter_metric_tag`：要报告的客观指标的名称。该名称必须与研究规范中指定的名称相同。
 - `metric_value`：要向超参数服务报告的客观指标的值。
 - `global_step`：epoch迭代的全局步骤，从0开始。

## 超参数调整模型

现在在Vertex上开始对您的自定义模型进行超参数调整。使用这个辅助函数 `create_hyperparameter_tuning_job`，它接受以下参数：

-`hpt_job`：超参数调整作业的规范。

辅助函数调用作业客户端服务的 `create_hyperparameter_tuning_job` 方法，参数如下：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`hyperparameter_tuning_job`：超参数调整作业的规范。

您将在`response`对象中显示返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义超参数调整作业的Vertex完全限定标识符。您需要保存这个标识符以便在后续步骤中使用。

`response.state`：自定义超参数调整作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获得您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
hpt_job_id = response.name
# The short numeric ID for the custom job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

## 获取超参数调整作业的信息

接下来，使用这个辅助函数`get_hyperparameter_tuning_job`，它接受以下参数：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

辅助函数调用作业客户服务的`get_hyperparameter_tuning_job`方法，参数为：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

如果你还记得，当调用`create_hyperparameter_tuning_job`方法时，在`response.name`字段中获取了超参数调整作业的 Vertex 完全限定标识符，并将该标识符保存在变量`hpt_job_id`中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

## 等待调整完成

调整上述模型的超参数可能需要长达20分钟的时间。

一旦您的模型调整完成，您可以通过从`end_time`减去`start_time`来计算调整模型所花费的实际时间。

对于您的模型，我们需要知道每次试验保存的模型位置，Python脚本保存在您本地的云存储桶中的`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

现在审查试验结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

最佳试验

现在看看哪个试验是最好的：

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

## 获取最佳模型

如果您使用了通过服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 桶中。现在从 Cloud Storage 桶加载它，然后可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 指定为 `MODEL_DIR`。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现如何。

### 加载评估数据

您将使用`tfds.datasets`中的`load()`方法加载IMDB电影评论测试（留存）数据。 这将返回一个包含两个元素的元组。 第一个元素是数据集，第二个是数据集的信息，其中包含预定义的词汇编码器。 编码器将单词转换为数值嵌入，这是在自定义训练脚本中预训练和使用的。

在训练模型时，您需要为文本设置一个固定的输入长度。 为了前向传递批次，相应的`tf.dataset`的`padded_batch()`属性被设置为将每个输入序列填充到相同的形状以进行批处理。

对于测试数据，您还需要相应地设置`padded_batch()`属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 进行模型评估

现在评估一下定制作业中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 管道
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage 存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME